## Instructions

### This code is used to convert the bone level segmentation masks determined by the YOLO model into bone level lines.

1. The data should be in the following structure

Directory structure:
```
images/
    1.jpg
    2.jpg
    ...
yolo_mask_outputs/
    7px/
        test/
            1.txt
            2.txt
        train/
            3.txt
            4.txt
        val/
            5.txt
            6.txt
    8px/
        test/
            1.txt
            2.txt
        train/
            3.txt
            4.txt
        val/
            5.txt
            6.txt
    ...
yolo_mask_outputs_denorm/ (will be generated with the same structure)
yolo_mask_outputs_denorm_lines/ (will be generated with the same structure. Contains the converted lines (converted using masks))

```


- `images/` should contain all the images in .jpg format. (Example: 1.jpg, 2.jpg)
- `bone_masks_normalized/` contains a directory structure as shown above. Inside a pixel size directory, it should contain a .txt file representing each image (can be inside `test`, `train` or `val` directory). Each file should contain the normalized bone level masks of the teeth in yolo format. (If `n` masks, there will be n lines. Each line contains space separated values. First value of the line will be the class id, which can be ignores, since we use only one class)

2. The denormalized masks will be saved to `yolo_mask_outputs_denorm/` directory.
3. The denormalized converted lines will be saved to `yolo_mask_outputs_denorm_lines/` directory.




# Denormalize data files

Use the first code block, if processing a single directory. If the directory contains test, train, val directory structure, use the second code block.

In [ ]:
import os
from PIL import Image

# Directories
images_dir = "images/"
bone_masks_normalized_dir = "yolo_mask_outputs"
bone_masks_denorm_dir = "denorm"

# Ensure the denormalized directory exists
os.makedirs(bone_masks_denorm_dir, exist_ok=True)

def process_txt_file(txt_file, input_dir, output_dir):
    # Construct the corresponding image file path
    image_file_name = os.path.splitext(txt_file)[0] + '.jpg'  # Assuming the images are in JPG format
    image_path = os.path.join(images_dir, image_file_name)

    # Check if the corresponding image file exists
    if os.path.exists(image_path):
        with Image.open(image_path) as img:
            width, height = img.size

        # Read the normalized txt file
        normalized_txt_path = os.path.join(input_dir, txt_file)
        with open(normalized_txt_path, 'r') as f:
            bone_details = f.readlines()

        # Denormalize coordinates and write to new file
        denorm_txt_path = os.path.join(output_dir, txt_file)
        with open(denorm_txt_path, 'w') as f:
            for line in bone_details:
                line = line[1:]  # remove the class id
                points = [float(p) for p in line.strip().split(" ")]
                denorm_points = []
                for i in range(0, len(points), 2):
                    x = points[i] * width
                    y = points[i+1] * height
                    denorm_points.extend([x, y])
                # Write the denormalized points to the new file
                f.write(','.join(map(str, denorm_points)) + '\n')
    else:
        print(f"Warning: No corresponding image found for txt file {txt_file}")

# Traverse through the directory structure
for px_folder in os.listdir(bone_masks_normalized_dir):
    px_path = os.path.join(bone_masks_normalized_dir, px_folder)
    if os.path.isdir(px_path):
        for split_folder in ['train', 'val', 'test']:
            input_split_path = os.path.join(px_path, split_folder)
            if os.path.exists(input_split_path):
                # Create corresponding output directory
                output_split_path = os.path.join(bone_masks_denorm_dir, px_folder, split_folder)
                os.makedirs(output_split_path, exist_ok=True)

                # Process txt files in this directory
                txt_files = [f for f in os.listdir(input_split_path) if f.endswith('.txt')]
                for txt_file in txt_files:
                    process_txt_file(txt_file, input_split_path, output_split_path)

print("Processing complete.")

Processing complete.


In [ ]:
!cp -r denorm/* yolo_mask_outputs_denorm/

# display to verify

In [ ]:
import os
from PIL import Image, ImageDraw

# Set the directory paths
images_dir = "images/"
bone_masks_dir = "denorm"
output_dir = "mask_outputs"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def process_file(filename, input_dir, output_dir):
    image_name = os.path.splitext(filename)[0] + ".jpg"
    image_path = os.path.join(images_dir, image_name)
    bone_mask_details_path = os.path.join(input_dir, filename)

    # Load image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Draw bone lines
    with open(bone_mask_details_path, "r") as f:
        bone_details = f.readlines()
        for line in bone_details:
            points = [float(p) for p in line.split(",")]
            points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
            draw.polygon(points, fill="red", width=2)

    # Save the image with bone masks
    output_path = os.path.join(output_dir, image_name)
    image.save(output_path)

# Traverse through the directory structure
for px_folder in os.listdir(bone_masks_dir):
    px_path = os.path.join(bone_masks_dir, px_folder)
    if os.path.isdir(px_path):
        for split_folder in ['train', 'val', 'test']:
            input_split_path = os.path.join(px_path, split_folder)
            if os.path.exists(input_split_path):
                # Create corresponding output directory
                output_split_path = os.path.join(output_dir, px_folder, split_folder)
                os.makedirs(output_split_path, exist_ok=True)

                # Process txt files in this directory
                txt_files = [f for f in os.listdir(input_split_path) if f.endswith('.txt')]
                for txt_file in txt_files:
                    process_file(txt_file, input_split_path, output_split_path)

print("Images with bone lines and masks saved in:", output_dir)

Images with bone lines and masks saved in: mask_outputs


In [ ]:
# FOR DISPLAYING GROUND TRUTH

import os
from PIL import Image, ImageDraw

# Set the directory paths
images_dir = "images/"
ground_truth_dir = "bone_lines_ground_truth"
bone_masks_dir = "denorm"
output_dir = "gt_outputs"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def process_file(image_name, ground_truth_path, output_dir):
    image_path = os.path.join(images_dir, image_name)

    # Load image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Draw bone lines from ground truth
    with open(ground_truth_path, "r") as f:
        bone_details = f.readlines()
        for line in bone_details:
            points = [float(p) for p in line.split(",")]
            points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
            draw.line(points, fill="red", width=4)

    # Save the image with bone masks
    output_path = os.path.join(output_dir, image_name)
    image.save(output_path)

# Traverse through the directory structure
for px_folder in os.listdir(bone_masks_dir):
    px_path = os.path.join(bone_masks_dir, px_folder)
    if os.path.isdir(px_path):
        for split_folder in ['train', 'val', 'test']:
            input_split_path = os.path.join(px_path, split_folder)
            if os.path.exists(input_split_path):
                # Create corresponding output directory
                output_split_path = os.path.join(output_dir, px_folder, split_folder)
                os.makedirs(output_split_path, exist_ok=True)

                # Process images and corresponding ground truth files in this directory
                image_files = [f for f in os.listdir(input_split_path) if f.endswith('.txt')]
                for image_file in image_files:
                    image_id = os.path.splitext(image_file)[0]
                    image_name = image_id + ".jpg"
                    txt_name = image_id + ".txt"
                    # if image_id != "1":
                    #    continue
                    ground_truth_path = os.path.join(ground_truth_dir, txt_name)
                    if os.path.exists(ground_truth_path):
                        process_file(image_name, ground_truth_path, output_split_path)

print("Images with ground truth bone lines saved in:", output_dir)


# Convert to line

In [ ]:
class Node:
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

def create_doubly_linked_list(points):
    if not points:
        return None

    head = Node(points[0])
    current = head

    for i in range(1, len(points)):
        new_node = Node(points[i])
        current.next = new_node
        new_node.prev = current
        current = new_node

    current.next = head
    head.prev = current

    return head

def adjust_head_to_leftmost(head):
    if not head:
        return None

    current = head
    leftmost = head
    while True:
        if current.data[0] < leftmost.data[0]:
            leftmost = current
        current = current.next
        if current == head:
            break

    return leftmost

def adjust_head_to_rightmost(head):
    if not head:
        return None

    current = head
    rightmost = head
    while True:
        if current.data[0] > rightmost.data[0]:
            rightmost = current
        current = current.next
        if current == head:
            break

    return rightmost

In [ ]:
import os
from PIL import Image, ImageDraw

# Set the directory paths
images_dir = "images/"
bone_masks_dir = "denorm"
bone_lines_output_dir = "lines_output"
output_dir = "output"

# Create output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(bone_lines_output_dir, exist_ok=True)

def process_file(filename, input_dir, output_image_dir, output_line_dir):
    image_name = os.path.splitext(filename)[0] + ".jpg"
    image_path = os.path.join(images_dir, image_name)
    bone_mask_details_path = os.path.join(input_dir, filename)

    # Load image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    converted_bones = []

    # Draw bone lines
    with open(bone_mask_details_path, "r") as f:
        bone_details = f.readlines()
        for line in bone_details:
            points = [float(p) for p in line.split(",")]
            points = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
            n = len(points)

            head = create_doubly_linked_list(points)

            head = adjust_head_to_leftmost(head)
            arr1 = []
            arr2 = []
            curr1 = head
            curr2 = head
            for i in range((n+1)//2):
                arr1.append(curr1.data)
                arr2.append(curr2.data)
                curr1 = curr1.next
                curr2 = curr2.prev

            head = adjust_head_to_rightmost(head)
            arr3 = []
            arr4 = []
            curr3 = head
            curr4 = head
            for i in range((n+1)//2):
                arr3.append(curr3.data)
                arr4.append(curr4.data)
                curr3 = curr3.next
                curr4 = curr4.prev

            arr3.reverse()
            arr4.reverse()

            mid1 = [((arr1[i][0] + arr2[i][0])/2, (arr1[i][1] + arr2[i][1])/2) for i in range(len(arr1))]
            mid2 = [((arr3[i][0] + arr4[i][0])/2, (arr3[i][1] + arr4[i][1])/2) for i in range(len(arr3))]
            mid = [((mid1[i][0] + mid2[i][0])/2, (mid1[i][1] + mid2[i][1])/2) for i in range(len(mid1))]

            midpoints_str = " ".join([f"{point[0]} {point[1]}" for point in mid])
            converted_bones.append(midpoints_str)

            draw.line(mid, fill="red", width=4)

    # save the converted bones, space separated
    bone_lines_output_path = os.path.join(output_line_dir, filename)
    with open(bone_lines_output_path, "w") as f:
        for bone in converted_bones:
            f.write(bone + "\n")

    # Save the image with bone masks
    output_path = os.path.join(output_image_dir, image_name)
    image.save(output_path)

# Traverse through the directory structure
for px_folder in os.listdir(bone_masks_dir):
    px_path = os.path.join(bone_masks_dir, px_folder)
    if os.path.isdir(px_path):
        for split_folder in ['train', 'val', 'test']:
            input_split_path = os.path.join(px_path, split_folder)
            if os.path.exists(input_split_path):
                # Create corresponding output directories
                output_image_split_path = os.path.join(output_dir, px_folder, split_folder)
                output_line_split_path = os.path.join(bone_lines_output_dir, px_folder, split_folder)
                os.makedirs(output_image_split_path, exist_ok=True)
                os.makedirs(output_line_split_path, exist_ok=True)

                # Process txt files in this directory
                txt_files = [f for f in os.listdir(input_split_path) if f.endswith('.txt')]
                for txt_file in txt_files:
                    process_file(txt_file, input_split_path, output_image_split_path, output_line_split_path)

print("Images with bone lines and masks saved in:", output_dir)
print("Converted bone lines saved in:", bone_lines_output_dir)

Images with bone lines and masks saved in: output
Converted bone lines saved in: lines_output


In [ ]:
!mkdir yolo_mask_outputs_denorm_lines
!cp -r lines_output/* yolo_mask_outputs_denorm_lines/

# PDF comparison

In [ ]:
!pip install reportlab

In [ ]:
!cp comparison_mask_to_line_conversion_prev_method.pdf comparison_mask_to_line_conversion_prev_method.pdf

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from PIL import Image
import os

# Define directories
mask_outputs_dir = 'gt_outputs'
line_outputs_dir = 'output'
pdfs_dir = 'yolo_mask_outputs_denorm_lines_pdfs_with_gt/'

def process_directory(mask_dir, line_dir, pdf_path):
    # Create a PDF using reportlab
    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter

    # List all images in the directories
    mask_outputs = [f for f in os.listdir(mask_dir) if f.endswith('.jpg')]
    line_outputs = [f for f in os.listdir(line_dir) if f.endswith('.jpg')]

    # Create a set of IDs from both directories
    mask_output_ids = {os.path.splitext(f)[0] for f in mask_outputs}
    line_output_ids = {os.path.splitext(f)[0] for f in line_outputs}

    # Find the common IDs
    common_ids = mask_output_ids & line_output_ids
    common_ids = sorted(common_ids)

    # Define the maximum number of rows per page
    rows_per_page = 3
    image_margin = 10  # Margin between images and page edges
    vertical_margin = 20  # Margin between rows

    # Calculate the maximum height for each image
    max_image_height = (height - vertical_margin * (rows_per_page + 1)) // rows_per_page
    max_image_width = (width // 2) - (2 * image_margin)

    current_row = 0

    for image_id in common_ids:
        mask_image_path = os.path.join(mask_dir, f"{image_id}.jpg")
        line_image_path = os.path.join(line_dir, f"{image_id}.jpg")

        # Open images to get their sizes
        mask_img = Image.open(mask_image_path)
        line_img = Image.open(line_image_path)

        # Resize images to fit the defined max dimensions (keeping aspect ratio)
        mask_img.thumbnail((max_image_width, max_image_height))
        line_img.thumbnail((max_image_width, max_image_height))

        # Calculate positions
        iopa_x = image_margin
        iopa_y = height - (current_row + 1) * (max_image_height + vertical_margin)
        angles_x = width // 2 + image_margin
        angles_y = height - (current_row + 1) * (max_image_height + vertical_margin)

        # Draw images on the PDF
        c.drawImage(mask_image_path, iopa_x, iopa_y, width=mask_img.width, height=mask_img.height)
        c.drawImage(line_image_path, angles_x, angles_y, width=line_img.width, height=line_img.height)

        # Increment row counter
        current_row += 1

        # If the maximum number of rows per page is reached, add a new page
        if current_row >= rows_per_page:
            c.showPage()
            current_row = 0

    # Save the PDF
    c.save()

# Traverse through the directory structure
for px_folder in os.listdir(mask_outputs_dir):
    px_path = os.path.join(mask_outputs_dir, px_folder)
    if os.path.isdir(px_path):
        for split_folder in ['train', 'val', 'test']:
            mask_split_path = os.path.join(px_path, split_folder)
            line_split_path = os.path.join(line_outputs_dir, px_folder, split_folder)
            if os.path.exists(mask_split_path) and os.path.exists(line_split_path):
                # Create the corresponding directory in pdfs_dir
                pdf_dir = os.path.join(pdfs_dir, px_folder, split_folder)
                os.makedirs(pdf_dir, exist_ok=True)

                # Create the PDF file path
                pdf_path = os.path.join(pdf_dir, f"comparison_{px_folder}_{split_folder}.pdf")

                # Process the directory and create the PDF
                process_directory(mask_split_path, line_split_path, pdf_path)
                print(f"PDF created and saved to {pdf_path}")

print("All PDFs have been created.")